## Transfer learning : Scaling up 

The point is to use transfer learning and extrapolate to the 101 food classes in the original dataset, trying to beat the accuracy of the original paper. 

At this point, i should use a GPU. GG WPPPPP

## Creating helper functions 

In previous notebooks, helper functions have been created. Let's use them !

In [1]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py

--2022-06-13 15:37:14--  https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py
Resolvendo github.com (github.com)... 140.82.121.3
Conectando con github.com (github.com)|140.82.121.3|:443... conectado.
Petición HTTP enviada, agardando unha resposta... 200 OK
Lonxitude: sen especificar [text/html]
Gardando en: «helper_functions.py.1»

helper_functions.py     [  <=>               ] 246,99K  1,13MB/s    in 0,2s    

2022-06-13 15:37:15 (1,13 MB/s) - gardouse «helper_functions.py.1» [252916]



In [2]:
# Import helper functions in our notebook
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,compare_historys,walk_through_dir

2022-06-13 15:37:15.474431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 15:37:15.474457: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## 101 Food Classes: working with less data

The goal is beat the state of the art, with less data, download it. 

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")



--2022-06-13 15:37:18--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolvendo storage.googleapis.com (storage.googleapis.com)... 142.250.201.80, 142.250.200.112, 216.58.215.176, ...
Conectando con storage.googleapis.com (storage.googleapis.com)|142.250.201.80|:443... conectado.
Petición HTTP enviada, agardando unha resposta... 200 OK
Lonxitude: 1625420029 (1,5G) [application/zip]
Gardando en: «101_food_classes_10_percent.zip.1»

s_10_percent.zip.1    2%[                    ]  40,48M  8,54MB/s    eta 3m 9s  ^C


KeyboardInterrupt: 

In [ ]:
walk_through_dir("101_food_classes_10_percent")

In [ ]:
#Setup data inputs
import tensorflow as tf

train_dir = "101_food_classes_10_percent/train"
test_dir = "101_food_classes_10_percent/test"

IMG_SIZE = (224,224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                               label_mode="categorical",
                                                               image_size=IMG_SIZE,
                                                               shuffle=False)
#Dont shuffle test data for prediction analysis

## Train a big dog model with TF on 10% of 1010 Food classes 

Here are the steps we're going to take: 
* Create a ModelCheckpoint callback
* Create a data augmentation layer
* Build a headless(no top layers) functional efficientnetb0 backbone model
* Compile our model 
* Feature extract for 5 epochs and validate on 15% of test data

In [ ]:
# Create checkpoint callback
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                        save_weights_only=True,
                                                        monitor="val_accuracy",
                                                        save_best_only=True)

In [ ]:
#Create a data augmentation layer to incorporate to the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

#Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    #preprocessing.Rescaling(1/255.) #Necessary for Resnet
],name="data_augmentation")


In [ ]:
# Setup the base model and freeze its layers(this will extract features)

base_model =tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model atchitecture with trainable top layers

inputs = layers.Input(shape=(224,224,3),name = "input_layer")
x = data_augmentation(inputs)#only in training
x = base_model(x,training=False) # to keep it frozen
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),activation="softmax",name="output")(x)
model = tf.keras.Model(inputs,outputs)


In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
             metrics = ["accuracy"])

history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                          epochs=5,
                                           validation_data=test_data,
                                          validation_steps=int(0.15*len(test_data)),
                                          callbacks = [checkpoint_callback])

In [ ]:
#Evaluate
feature_extraction_results = model.evaluate(test_data)
feature_extraction_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

## Fine tuning 

In [ ]:
# Unfreeze all of the layers in the base model 
base_model.trainable = True

#Refreeze except 5 last 
for layer in base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
#Recompile  and lower learning rate (good practise with fine tuning)
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
             optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
             metrics = ["accuracy"])

In [ ]:
# What layers are trainbable :
for layer in model.layers:
    print(layer.name,layer.trainable)

In [ ]:
for layer_number,layer in enumerate(base_model.layers):
    print(layer_number,layer.name,layer.trainable)

In [ ]:
# Fine- tune for more 5 epochs 
fine_tune_epochs = 10 

history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                    epochs = fine_tune_epochs,
                                                    validation_data=test_data,
                                                    validation_steps=int(0.15*len(test_data)),
                                                    initial_epoch = history_all_classes_10_percent.epoch[-1]
                                                                        )

In [ ]:
#Evaluate
all_classes_10_percent_fine_tune_results = model.evaluate(test_data)
all_classes_10_percent_fine_tune_results

In [ ]:
compare_historys(history_all_classes_10_percent,history_all_classes_10_percent_fine_tune)

## Saving and loading our model 

To use it in an external app, need no export it 

In [ ]:
import os
# Save fine-tuning model 
model.save(filepath = os.getcwd())

In [ ]:
# Load an evaluate saved model 
import os
loaded_model = tf.keras.models.load_model(os.getcwd())

In [ ]:
# Evaluate loaded model and compare performance to pre-saved model
loaded_model_results = loaded_model.evaluate(test_data)
loaded_model_results

In [ ]:
all_classes_10_percent_fine_tune_results

### Evaluate the model on predictions

Make predictions, visualize them and check the most wrong.

In [ ]:
import tensorflow as tf
#Download pretrained model to compare with bosses
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [ ]:
unzip_data("06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
model = tf.keras.models.load_model("06_101_food_class_10_percent_saved_big_dog_model/")

In [ ]:
#Evaluate loaded model on test data
results_downloaded_model = model.evaluate(test_data)
results_downloaded_model

In [ ]:
model.summary()

## Make predictions with trained model

In [ ]:
preds_probs = model.predict(test_data, verbose = 1)

In [ ]:
#pred classes for each label 
pred_classes = preds_probs.argmax(axis=1)

The model outputs a prediction probability for each class

In [ ]:
# To get out test labels we need to unravel our test_data
y_labels = []
for images,labels in test_data.unbatch():
    y_labels.append(labels.numpy().argmax())

We have to chech if our model predictions array is in the same order as this one. One way to do this is measure the accuracy score. 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
sklearn_accuracy = accuracy_score(y_labels,pred_classes)

In [ ]:
import numpy as np
np.isclose(sklearn_accuracy,all_classes_10_percent_fine_tune_results,10e-4)

### Confusion matrix


In [ ]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,compare_historys,walk_through_dir
from helper_functions import make_confusion_matrix

In [ ]:
make_confusion_matrix(y_labels,pred_classes,test_data.class_names,
                     figsize=(100,100),
                     text_size = 20)

In [ ]:
## New metric evaluations, this time, a classification report from sklearn 
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true=y_labels,y_pred=pred_classes))

In [ ]:
#Get a dictionary of the classification report

In [ ]:
classification_report_dict = classification_report(y_true=y_labels,y_pred=pred_classes,output_dict=True)

In [ ]:
# Create a plot of F1-scores
class_f1_scores = {}
for k,v in classification_report_dict.items():
    if k == "accuracy":
        break
    else:
        class_f1_scores[test_data.class_names[int(k)]] = v["f1-score"]

In [ ]:
# Create dataframe
import pandas as pd
f1_scores = pd.DataFrame({"class_names": list(class_f1_scores.keys()),
                         "f1-score":list(class_f1_scores.values())}
                        ).sort_values(by="f1-score")
f1_scores.sample(5)

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(12,25))
scores = ax.barh(range(len(f1_scores)),f1_scores["f1-score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(f1_scores["class_names"]);
ax.set_title("F1-scores for 101 foods ");


## Visualizing predictions on test images

To visualize our model predictions on our own images, we need a function to load and preprocess images 

In [ ]:
# Create that function 
def load_and_prep_image(filename,img_shape=224,scale=True):
    """Reads in an image from filenae, turns into a tensor and 
    reshapes into specified shape.
    """
    #Read
    img = tf.io.read_file(filename)
    #Convert to tensor
    img = tf.io.decode_image(img,channels=3)
    #Resize
    img = tf.image.resize(img,[img_shape,img_shape])
    
    #Scale
    if scale:
        return img/255.0
    else:
        return img

 Write the code to visualize images, target label and predictions

In [ ]:
#Make preds on a series of random images
import os 
import random 

plt.figure(figsize=(17,10))
for i in range(3):
    class_name = random.choice(test_data.class_names)
    filename = random.choice(os.listdir(test_dir+"/"+class_name))
    filepath = test_dir+ "/" + class_name + "/" + filename
    #Load the image and make predictions
    img = load_and_prep_image(filepath,scale=False)
    pred_prob = model.predict(tf.expand_dims(img,axis=0))
    pred_class = test_data.class_names[pred_prob.argmax()]
    #Plot the image
    plt.subplot(1,3,1+i)
    plt.imshow(img/255.)
    if class_name == pred_class:
        title_color = "g"
    else: 
        title_color = "r"
    plt.title(f"actual: {class_name}, pred: {pred_class}",color=title_color)
    plt.axis(False)

## Finding the most wrong predictions 

In [ ]:
filepaths = []
for filepath in tf.data.Dataset.list_files(test_dir+"/*/*.jpg",shuffle=False):
    filepaths.append(filepath.numpy())

In [ ]:
#Create a dataframe of different parameters for our test images
import pandas as pd
pred_df = pd.DataFrame({"img_path":filepaths,
                        "y_true":y_labels,
                        "y_pred":pred_classes,
                        "pred_conf":preds_probs.max(axis=1),
                        "y_true_classname":[test_data.class_names[i] for i in y_labels],
                        "y_pred_classname":[test_data.class_names[i] for i in pred_classes]
                       })

In [ ]:
#Find out the most wrong predictions

pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head(10)

In [ ]:
#Sort our dataframe to make the most wrong go up 
top_100_wrong = pred_df[pred_df["pred_correct"] == False].sort_values(by="pred_conf",ascending=False)[:100]

In [ ]:
top_100_wrong.head(10)

In [ ]:
# Visualize top_100_wrong
images_to_view = 9 
start_index = 30
plt.figure(figsize=(15,10))
for i,row in enumerate(top_100_wrong[start_index:start_index+images_to_view].itertuples()):
    plt.subplot(3,3,i+1)
    img = load_and_prep_image(row[1],scale=False)
    plt.imshow(img/255.)
    plt.axis(False)
    _,_,_,_,pred_prob,y_true_classname,y_pred_classname,_ = row
    plt.title(f"actual:{y_true_classname}, pred: {y_pred_classname},\n prob {pred_prob:.3f}")

### Predict on custom images 

In [ ]:
#Get custom images
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

In [ ]:
unzip_data("custom_food_images.zip")

In [ ]:
custom_food_images = ["custom_food_images/"+img_path for img_path in os.listdir("custom_food_images/")]

In [ ]:
#Make predictions and plot custom food images
for img in custom_food_images:
    img = load_and_prep_image(img,scale=False)
    pred_prob = model.predict(tf.expand_dims(img,axis=0))
    pred_class = test_data.class_names[pred_prob.argmax()]
    plt.figure()
    plt.imshow(img/255.)
    plt.title(f"pred:{pred_class},prob:{pred_prob.max():.2f}")